# Mistral 7B F1 QA Evaluation

This notebook evaluates Mistral 7B on the F1 QA dataset with multiple choice questions.

It loads 500 JSON files (1500 QA pairs total), formats them as multiple choice questions, and evaluates the model's accuracy.


In [ ]:
# Install required packages with latest versions
%pip install transformers>=4.40.0 torch>=2.0.0 accelerate>=0.20.0 huggingface_hub>=0.20.0 tqdm


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
print("✅ Google Drive mounted successfully!")

# Hugging Face Authentication for Mistral model
from huggingface_hub import login
import os

from google.colab import userdata
# Check if HF token is available
if userdata.get('HF_TOKEN'):
    login(token=userdata.get('HF_TOKEN'))
    print("✅ Hugging Face authenticated via environment variable")
else:
    print("⚠️  Hugging Face token not found in environment variables")
    print("   Please set your HF_TOKEN in Colab secrets or run:")
    print("   from huggingface_hub import login")
    print("   login()  # This will prompt for your token")
    print("   Or set HF_TOKEN in Colab secrets (recommended)")


Mounted at /content/drive
✅ Google Drive mounted successfully!
✅ Hugging Face authenticated via environment variable


In [ ]:
# Import utilities with latest transformers syntax
import sys
sys.path.append('/content')
from f1_qa_utils import load_qa_dataset, format_multiple_choice, extract_answer_letter, extract_justification, parse_json_response

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
from tqdm import tqdm
import json
from pathlib import Path
import time
import gc
from typing import List, Dict, Any
import warnings
warnings.filterwarnings('ignore')


## Configuration

Set paths and model to evaluate.


In [ ]:
# Configuration for Mistral 7B
MODEL_ID = "mistralai/Mistral-7B-v0.1"
MODEL_NAME = "mistral-7b"

DATASET_PATH = "/content/drive/MyDrive/CS6220_Project/dataset/"
RESULTS_PATH = "/content/drive/MyDrive/CS6220_Project/results"

# Create results directory if needed
Path(RESULTS_PATH).mkdir(parents=True, exist_ok=True)

print(f"✅ Configuration loaded:")
print(f"   Model: {MODEL_NAME}")
print(f"   Model ID: {MODEL_ID}")
print(f"   Dataset: {DATASET_PATH}")
print(f"   Results: {RESULTS_PATH}")
print(f"   📝 Note: Model will be loaded directly from Hugging Face Hub")
print(f"   ")


✅ Configuration loaded:
   Model: mistral-7b
   Model ID: mistralai/Mistral-7B-v0.1
   Dataset: /content/drive/MyDrive/CS6220_Project/dataset/
   Results: /content/drive/MyDrive/CS6220_Project/results
   📝 Note: Model will be loaded directly from Hugging Face Hub
   


## Load QA Dataset


In [ ]:
# Load all QA pairs
qa_dataset = load_qa_dataset(DATASET_PATH)

print(f"📊 Dataset Statistics:")
print(f"   Total QA pairs: {len(qa_dataset)}")
print(f"   Expected: 1500 (500 files × 3 QA pairs each)")

# Show example
if qa_dataset:
    print(f"\n📝 Example QA pair:")
    example = qa_dataset[0]
    print(f"   Question: {example['rephrased_question']}")
    print(f"   Correct Answer: {example['ground_truth_correct_option']}")
    print(f"   Options: {example['options']}")

    # Show example of formatted prompt
    print(f"\n📝 Example formatted prompt:")
    prompt = format_multiple_choice(
        example['rephrased_question'],
        example['options']
    )
    print(f"   Prompt preview: {prompt[:200]}...")


✅ Loaded 1500 QA pairs from /content/drive/MyDrive/CS6220_Project/dataset
📊 Dataset Statistics:
   Total QA pairs: 1500
   Expected: 1500 (500 files × 3 QA pairs each)

📝 Example QA pair:
   Question: Which driver secured a double victory at the US Grand Prix held in Austin?
   Correct Answer: C
   Options: {'A': 'Sergio Pérez', 'B': 'Lewis Hamilton', 'C': 'Max Verstappen', 'D': 'Charles Leclerc'}

📝 Example formatted prompt:
   Prompt preview: You are an AI assistant specializing in Formula 1.

    Question: Which driver secured a double victory at the US Grand Prix held in Austin?

    A) Sergio Pérez
    B) Lewis Hamilton
    C) Max Verst...


## Model Evaluator Class


In [ ]:
class F1QAEvaluator:
    """Evaluator for F1 QA models with latest transformers syntax."""

    def __init__(self, model_id: str, model_name: str):
        self.model_id = model_id
        self.model_name = model_name
        self.model = None
        self.tokenizer = None
        self.device = "cuda" if torch.cuda.is_available() else "cpu"

    def load_model(self):
        """Load model and tokenizer directly from Hugging Face Hub."""
        try:
            print(f"🔄 Loading {self.model_name} from {self.model_id}...")

# Load tokenizer with latest syntax
            self.tokenizer = AutoTokenizer.from_pretrained(
                self.model_id,
                trust_remote_code=True,
                use_fast=True
            )

            # Set padding token if not set
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token

            # Load model with latest syntax and optimizations
            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_id,
                trust_remote_code=True,
                torch_dtype=torch.float16 if self.device == "cuda" else torch.float32,
                device_map="auto" if self.device == "cuda" else None,
                attn_implementation="eager"  # Use eager attention for better compatibility
            )

            # Move to device if not using device_map
            if self.device == "cuda" and not hasattr(self.model, 'hf_device_map'):
                self.model = self.model.to(self.device)

            print(f"✅ {self.model_name} loaded successfully on {self.device}")
            return True

        except Exception as e:
            print(f"❌ Failed to load {self.model_name}: {e}")
            return False

    def generate_response(self, prompt: str, max_new_tokens: int = 5) -> str:
        """Generate response using latest transformers generation syntax."""
        try:
            # Tokenize with proper attention mask handling
            inputs = self.tokenizer(
                prompt,
                return_tensors="pt",
                truncation=True,
                max_length=512,
                padding=True
            )

            if self.device == "cuda":
                inputs = {k: v.to(self.device) for k, v in inputs.items()}

            # Create generation config with latest syntax
            generation_config = GenerationConfig(
                max_new_tokens=max_new_tokens,
                do_sample=True,
                temperature=0.3,
                top_p=0.9,
                pad_token_id=self.tokenizer.eos_token_id,
                eos_token_id=self.tokenizer.eos_token_id,
                repetition_penalty=1.1,
                use_cache=True,  # Llama works well with cache enabled
                return_dict_in_generate=True,
                output_scores=False
            )

            with torch.no_grad():
                outputs = self.model.generate(
                    inputs["input_ids"],
                    attention_mask=inputs["attention_mask"],
                    generation_config=generation_config
                )

            # Check if outputs is valid
            if outputs is None or not hasattr(outputs, 'sequences'):
                return ""

            # Decode only the new tokens (excluding the input prompt)
            input_length = inputs["input_ids"].shape[1]
            if len(outputs.sequences[0]) > input_length:
                new_tokens = outputs.sequences[0][input_length:]
                response = self.tokenizer.decode(new_tokens, skip_special_tokens=True).strip()
            else:
                # If no new tokens generated, return empty string
                response = ""

            return response

        except Exception as e:
            print(f"❌ Error generating response: {e}")
            return ""

    def evaluate_dataset(self, qa_dataset: List[Dict]) -> Dict:
        """Evaluate model on entire dataset."""
        results = {
            'model_name': self.model_name,
            'total_questions': len(qa_dataset),
            'correct': 0,
            'incorrect': 0,
            'invalid': 0,
            'accuracy': 0.0,
            'details': []
        }

        print(f"\n🚀 Evaluating {self.model_name} on {len(qa_dataset)} questions...")

        for i, qa_pair in enumerate(tqdm(qa_dataset, desc=f"Evaluating {self.model_name}")):
            question = qa_pair['rephrased_question']
            correct_letter = qa_pair['ground_truth_correct_option']
            options = qa_pair['options']

            # Format as multiple choice
            prompt = format_multiple_choice(question, options)

            # Generate response
            response = self.generate_response(prompt)

            print(f"RESPONSE IS: {response}")


            # Extract answer letter and justification
            predicted_letter = extract_answer_letter(response)
            #justification = extract_justification(response)

            print(f"EXTRACTED ANSWER: {predicted_letter}")

            # Check correctness
            is_correct = predicted_letter == correct_letter
            is_invalid = predicted_letter == ""

            if is_correct:
                results['correct'] += 1
            elif is_invalid:
                results['invalid'] += 1
            else:
                results['incorrect'] += 1

            # Store details
            results['details'].append({
                'question': question,
                'correct_letter': correct_letter,
                'predicted_letter': predicted_letter,
                'is_correct': is_correct,
                'is_invalid': is_invalid,
                'response': response,
                #'justification': justification,
                'prompt': prompt
            })

        # Calculate accuracy
        valid_responses = results['total_questions'] - results['invalid']
        if valid_responses > 0:
            results['accuracy'] = results['correct'] / valid_responses

        print(f"✅ {self.model_name} evaluation complete:")
        print(f"   Correct: {results['correct']}")
        print(f"   Incorrect: {results['incorrect']}")
        print(f"   Invalid: {results['invalid']}")
        print(f"   Accuracy: {results['accuracy']:.2%}")

        return results

    def cleanup(self):
        """Clear model from memory."""
        del self.model
        del self.tokenizer
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        print(f"🧹 Cleaned up {self.model_name} from memory")


## Run Evaluation


In [ ]:
# Evaluate Mistral 7B
print(f"\n{'='*60}")
print(f"🤖 Evaluating Model: {MODEL_NAME}")
print(f"{'='*60}")

evaluator = F1QAEvaluator(MODEL_ID, MODEL_NAME)

if evaluator.load_model():
    results = evaluator.evaluate_dataset(qa_dataset)
    evaluator.cleanup()

    # Save results to JSON
    output_file = Path(RESULTS_PATH) / f"mistral_7b_evaluation_results_{int(time.time())}.json"

    results_summary = {
        'metadata': {
            'timestamp': time.strftime('%Y-%m-%d %H:%M:%S'),
            'model_name': MODEL_NAME,
            'model_id': MODEL_ID,
            'total_qa_pairs': len(qa_dataset),
            'device': evaluator.device
        },
        'results': results
    }

    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(results_summary, f, indent=2, ensure_ascii=False)

    print(f"\n💾 Results saved to: {output_file}")

    # Print summary
    print(f"\n{'='*60}")
    print(f"📊 MISTRAL 7B EVALUATION SUMMARY")
    print(f"{'='*60}")
    print(f"   Accuracy: {results['accuracy']:.2%}")
    print(f"   Correct: {results['correct']}/{results['total_questions']}")
    print(f"   Incorrect: {results['incorrect']}")
    print(f"   Invalid: {results['invalid']}")
    print(f"\n💾 Full results saved to: {output_file}")

else:
    print(f"❌ Failed to load {MODEL_NAME}")



🤖 Evaluating Model: mistral-7b
🔄 Loading mistral-7b from mistralai/Mistral-7B-v0.1...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ mistral-7b loaded successfully on cuda

🚀 Evaluating mistral-7b on 1500 questions...


Evaluating mistral-7b:   0%|          | 1/1500 [00:08<3:20:24,  8.02s/it]

RESPONSE IS: Answer:
EXTRACTED ANSWER: 


...

Evaluating mistral-7b: 100%|█████████▉| 1499/1500 [2:28:25<00:05,  5.37s/it]

RESPONSE IS: C
EXTRACTED ANSWER: C


Evaluating mistral-7b: 100%|██████████| 1500/1500 [2:28:33<00:00,  5.94s/it]

RESPONSE IS: This is
EXTRACTED ANSWER: 
✅ mistral-7b evaluation complete:
   Correct: 644
   Incorrect: 754
   Invalid: 102
   Accuracy: 46.07%


🧹 Cleaned up mistral-7b from memory

💾 Results saved to: /content/drive/MyDrive/CS6220_Project/results/mistral_7b_evaluation_results_1763336927.json

📊 MISTRAL 7B EVALUATION SUMMARY
   Accuracy: 46.07%
   Correct: 644/1500
   Incorrect: 754
   Invalid: 102

💾 Full results saved to: /content/drive/MyDrive/CS6220_Project/results/mistral_7b_evaluation_results_1763336927.json
